In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import lightgbm as lgbm
import gc
import xgboost as xgb
import pickle as pickle


from catboost import CatBoostRegressor
import dask.dataframe as dd
from sklearn.model_selection import KFold
from itertools import product

In [3]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [4]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
sales_train = sales_train[sales_train['year'] != 2013]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [5]:
train_item_ids = sales_train['item_id'].unique()
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))

In [6]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    dbn_combos = list(product(sales.shop_id.unique(), sales.item_id.unique(), [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [7]:
len(all_combos)

6425094

In [8]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"y"})

In [9]:
#

In [10]:
training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['y'] = training['y'].clip(0,20)
training['y'] = training['y'].astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

In [11]:
def get_mean_encoding(df, group_cols, target):
    cumsum = df.groupby(group_cols)[target].cumsum() - df[target]
    cumcnt = df.groupby(group_cols).cumcount()
    return cumsum/cumcnt

training['item_me'] = pd.to_numeric(get_mean_encoding(training, ['item_id'], 'y'), downcast='float')
training['shop_me'] = pd.to_numeric(get_mean_encoding(training, ['shop_id'], 'y'), downcast='float')
training['category_me'] = pd.to_numeric(get_mean_encoding(training, ['item_category_id'], 'y'), downcast='float')
training['shop_category_me'] = pd.to_numeric(get_mean_encoding(training, ['shop_id', 'item_category_id'], 'y'), downcast='float')
training['shop_item_me'] = pd.to_numeric(get_mean_encoding(training, ['shop_id', 'item_id'], 'y'), downcast='float')

training.fillna(0,inplace=True)

In [12]:
cols = ['item_id','date_block_num']
item_block_units = sales_train.groupby(cols,as_index=False)['item_cnt_day'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_day':'item_block_units'})

training = training.merge(item_block_units, on=cols, how='left').fillna(0)
del item_block_units
gc.collect()
#
cols = ['shop_id','date_block_num']
shop_block_units = sales_train.groupby(cols,as_index=False)['item_cnt_day'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_day':'shop_block_units'})

training = training.merge(shop_block_units, on=cols, how='left').fillna(0)
del shop_block_units
gc.collect()
#
cols = ['item_category_id','date_block_num']
cat_block_units = sales_train.groupby(cols,as_index=False)['item_cnt_day'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_day':'cat_block_units'})

training = training.merge(cat_block_units, on=cols, how='left').fillna(0)
del cat_block_units
gc.collect()
#

cols = ['shop_id', 'item_category_id','date_block_num']
shop_cat_block_units = sales_train.groupby(cols,as_index=False)['item_cnt_day'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_day':'shop_cat_block_units'})

training = training.merge(shop_cat_block_units, on=cols, how='left').fillna(0)
del shop_cat_block_units
gc.collect()
#
cols = ['shop_id', 'item_id','date_block_num']
shop_item_block_units = sales_train.groupby(cols,as_index=False)['item_cnt_day'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_day':'shop_item_block_units'})

training = training.merge(shop_item_block_units, on=cols, how='left').fillna(0)
del shop_item_block_units
gc.collect()

63

In [13]:
number_of_items = sales_train['item_id'].nunique()
print("number_of_items:", number_of_items)
number_of_categories = sales_train['item_category_id'].nunique()
print("number_of_categories:", number_of_categories)
number_of_shops = sales_train['shop_id'].nunique()
print("number_of_shops:", number_of_shops)
number_of_days = 365 + 365 - 30 - 31
print("number_of_days:", number_of_days)
number_of_blocks = sales_train['date_block_num'].nunique()
print("number_of_blocks:", number_of_blocks)
total_sales = sales_train['item_cnt_day'].sum()
print("total_sales:", total_sales)
average_price = sales_train['item_price'].mean()
print("average_price:", average_price)

number_of_items: 17054
number_of_categories: 79
number_of_shops: 55
number_of_days: 669
number_of_blocks: 22
total_sales: 2085473.0
average_price: 1015.5023073772021


In [14]:
training['item_max_units_block'] = training.groupby(['item_id'])['item_block_units']\
        .transform(np.max)
training['item_min_units_block'] = training.groupby(['item_id'])['item_block_units']\
        .transform(np.min)
training['item_minmax_mean'] = training[['item_max_units_block', 'item_min_units_block']].mean(axis=1)

for q in [0.25,0.50,0.75]:
    name = 'item_minmax_q' + str(q)
    training[name] =  training[['item_min_units_block','item_max_units_block']].quantile(q,axis=1)


#
training['shop_units'] = training.groupby(['date_block_num'])['shop_block_units'].transform(np.sum)
training['shop_max_units_block'] = training.groupby(['shop_id'])['shop_block_units']\
        .transform(np.max)
training['shop_min_units_block'] = training.groupby(['shop_id'])['shop_block_units']\
        .transform(np.min)
training['shop_minmax_mean'] = training[['shop_max_units_block', 'shop_min_units_block']].mean(axis=1)
for q in [0.25,0.50,0.75]:
    name = 'shop_minmax_q' + str(q)
    training[name] =  training[['shop_min_units_block','shop_max_units_block']].quantile(q,axis=1)

#
training['cat_units'] = training.groupby(['date_block_num'])['cat_block_units'].transform(np.sum)
training['cat_max_units_block'] = training.groupby(['item_category_id'])['cat_block_units']\
        .transform(np.max)
training['cat_min_units_block'] = training.groupby(['item_category_id'])['cat_block_units']\
        .transform(np.min)
training['cat_minmax_mean'] = training[['cat_max_units_block', 'cat_min_units_block']].mean(axis=1)
#
training['shop_cat_units'] = training.groupby(['date_block_num'])['shop_cat_block_units'].transform(np.sum)
training['shop_cat_max_units_block'] = training.groupby(['shop_id', 'item_category_id'])['shop_cat_block_units']\
        .transform(np.max)
training['shop_cat_min_units_block'] = training.groupby(['shop_id', 'item_category_id'])['shop_cat_block_units']\
        .transform(np.min)
training['shop_cat_minmax_mean'] = training[['shop_cat_max_units_block', 'shop_cat_min_units_block']].mean(axis=1)
#
  training['shop_item_max_units_block'] = training.groupby(['shop_id', 'item_id'])['shop_item_block_units']\
        .transform(np.max)
training['shop_item_min_units_block'] = training.groupby(['shop_id', 'item_id'])['shop_item_block_units']\
        .transform(np.min)
training['shop_item_minmax_mean'] = training[['shop_item_max_units_block', 'shop_item_min_units_block']].mean(axis=1)

In [45]:
training['shop_item_block_units_mean'] = training.groupby(['item_id', 'date_block_num'])['y'].transform(np.mean)

In [247]:
mean_price_block = sales_train.groupby(['item_id','date_block_num'],as_index=False)['item_price']\
    .mean().rename(columns={'item_price':'mean_price_block'})
training = training.merge(mean_price_block, on=['item_id','date_block_num'], how='left')
training['item_max_price'] = training['item_id'].map(sales_train.groupby(['item_id'])['item_price'].max())
training['item_min_price'] = training['item_id'].map(sales_train.groupby(['item_id'])['item_price'].min())



for q in [0.25,0.50,0.75]:
    name = 'item_price_minmax_q' + str(q)
    training[name] =  training[['item_min_price','item_max_price']].quantile(q,axis=1)

In [252]:
#training['price_1'] = training['mean_price_block'] * 100 / training['item_max_price']

In [15]:
training['item_share_of_total_units'] = training['item_units'] * 100 / total_sales
training['category_share_of_total_units'] = training['cat_units'] * 100 / total_sales
training['shop_share_of_units'] = training['shop_units'] * 100 / total_sales
training['shop_item_share_of_total_units'] = training['shop_item_units'] * 100\
                        / total_sales
training['shop_item_share_of_shop_units'] = training['shop_item_units'] * 100\
                        / training['shop_units']

In [16]:
#training['item_share_of_total_shop_units'] 

shop_item_units = sales_train.groupby(['shop_id', 'item_id'])['item_cnt_day'].sum().reset_index()\
        .rename(columns={'item_cnt_day':'shop_item_units'})
training = training.merge(shop_item_units, on=['shop_id', 'item_id'], how='left')
training['item_share_of_shop_units'] = training['shop_item_units_y'] * 100 / training['shop_units']

In [309]:
len(training)

6425094

In [17]:
rolls = [2,3,6,12]
cols = ['item_id','date_block_num']

for roll in rolls:
    print(roll)
    roll_name = "item_block_units_rolling_" + str(roll)
    roll_name_tmp = roll_name + "_tmp"  
    
    item_block_units_rolling_temp = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_id'],as_index=False)\
        ['item_block_units'].rolling(roll,min_periods=2).mean().reset_index()\
        .rename(columns={'item_block_units':roll_name_tmp})\
        [['item_id','date_block_num',roll_name_tmp]]
    
    training = training.merge(item_block_units_rolling_temp, on=cols, how='left')
    del item_block_units_rolling_temp
    gc.collect()

    item_block_units_rolling = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_id'],as_index=False)\
        [roll_name_tmp].shift(1)\
        .rename(columns={roll_name_tmp:roll_name}).reset_index()

    training = training.merge(item_block_units_rolling, on=cols, how='left')
    training.drop(columns=[roll_name_tmp], inplace=True)
    del item_block_units_rolling
    gc.collect()

2
3
6
12


In [48]:
rolls = [2,3,6]
cols = ['item_id','date_block_num']

for roll in rolls:
    print(roll)
    roll_name = "shop_item_block_units_mean_rolling_" + str(roll)
    roll_name_tmp = roll_name + "_tmp"
    
    shop_item_block_units_mean_temp = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_id'],as_index=False)\
        ['shop_item_block_units_mean'].rolling(roll,min_periods=2).mean().reset_index()\
        .rename(columns={'shop_item_block_units_mean':roll_name_tmp})\
        [['item_id','date_block_num',roll_name_tmp]]
    
    training = training.merge(shop_item_block_units_mean_temp, on=cols, how='left')
    del shop_item_block_units_mean_temp
    gc.collect()

    shop_item_block_units_mean = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_id'],as_index=False)\
        [roll_name_tmp].shift(1)\
        .rename(columns={roll_name_tmp:roll_name}).reset_index()

    training = training.merge(shop_item_block_units_mean, on=cols, how='left')
    training.drop(columns=[roll_name_tmp], inplace=True)
    del shop_item_block_units_mean
    gc.collect()

2
3
6


In [24]:
rolls = [3,6,12]
cols = ['shop_id','date_block_num']

for roll in rolls:
    print(roll)
    roll_name = "shop_block_units_rolling_" + str(roll)
    roll_name_tmp = roll_name + "_tmp"
    
    shop_block_units_rolling_temp = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['shop_id'],as_index=False)\
        ['shop_block_units'].rolling(roll,min_periods=2).mean().reset_index()\
        .rename(columns={'shop_block_units':roll_name_tmp})\
        [['shop_id','date_block_num',roll_name_tmp]]
    
    training = training.merge(shop_block_units_rolling_temp, on=cols, how='left')
    del shop_block_units_rolling_temp
    gc.collect()

    shop_block_units_rolling = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['shop_id'],as_index=False)\
        [roll_name_tmp].shift(1)\
        .rename(columns={roll_name_tmp:roll_name}).reset_index()

    training = training.merge(shop_block_units_rolling, on=cols, how='left')
    training.drop(columns=[roll_name_tmp], inplace=True)
    del shop_block_units_rolling
    gc.collect()

3
6
12


In [ ]:
rolls = [3,6,12]
cols = ['item_category_id', 'date_block_num']

for roll in rolls:
    print(roll)
    roll_name = "cat_block_units_rolling_" + str(roll)
    roll_name_tmp = roll_name + "_tmp"
    
    cat_block_units_rolling_temp = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_category_id'],as_index=False)\
        ['cat_block_units'].rolling(roll,min_periods=2).mean().reset_index()\
        .rename(columns={'cat_block_units':roll_name_tmp})\
        [['item_category_id','date_block_num',roll_name_tmp]]
    
    training = training.merge(cat_block_units_rolling_temp, on=cols, how='left')
    del cat_block_units_rolling_temp
    gc.collect()
    
    cat_block_units_rolling = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_category_id'],as_index=False)\
        [roll_name_tmp].shift(1)\
        .rename(columns={roll_name_tmp:roll_name}).reset_index()

    training = training.merge(cat_block_units_rolling, on=cols, how='left')
    training.drop(columns=[roll_name_tmp], inplace=True)
    del cat_block_units_rolling
    gc.collect()

In [ ]:
rolls = [3,6,12]
cols = ['shop_id', 'item_category_id', 'date_block_num']

for roll in rolls:
    print(roll)
    roll_name = "shop_cat_block_units_rolling_" + str(roll)
    roll_name_tmp = roll_name + "_tmp"
    
    shop_cat_block_units_rolling_temp = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['shop_id', 'item_category_id'],as_index=False)\
        ['shop_cat_block_units'].rolling(roll,min_periods=2).mean().reset_index()\
        .rename(columns={'shop_cat_block_units':roll_name_tmp})\
        [['shop_id', 'item_category_id','date_block_num',roll_name_tmp]]
    
    training = training.merge(shop_cat_block_units_rolling_temp, on=cols, how='left')
    del shop_cat_block_units_rolling_temp
    gc.collect()
    
    shop_cat_block_units_rolling = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['shop_id', 'item_category_id'],as_index=False)\
        [roll_name_tmp].shift(1)\
        .rename(columns={roll_name_tmp:roll_name}).reset_index()

    training = training.merge(shop_cat_block_units_rolling, on=cols, how='left')
    training.drop(columns=[roll_name_tmp], inplace=True)
    del shop_cat_block_units_rolling
    gc.collect()

In [ ]:
training.fillna(0, inplace=True)

In [18]:
lags = [1,2,3,6,12]

cols = ['item_id', 'date_block_num']


for lag in lags:
    print(lag)
    lag_name = "item_block_units_lag_" + str(lag)
    item_block_units_lag = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_id'],as_index=False)\
        ['item_block_units'].shift(lag)\
        .rename(columns={'item_block_units':lag_name}).reset_index()

    training = training.merge(item_block_units_lag, on=cols, how='left')
    del item_block_units_lag
    gc.collect()

1
2
3
6
12


In [60]:
lags = [1,2,3,6,12]

cols = ['item_id', 'date_block_num']


for lag in lags:
    print(lag)
    lag_name = "shop_item_block_units_mean_lag_" + str(lag)
    shop_item_block_units_mean = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_id'],as_index=False)\
        ['shop_item_block_units_mean'].shift(lag)\
        .rename(columns={'shop_item_block_units_mean':lag_name}).reset_index()

    training = training.merge(shop_item_block_units_mean, on=cols, how='left')
    del item_block_units_lag
    gc.collect()

1


NameError: name 'item_block_units_lag' is not defined

In [23]:
lags = [1,2,3,6,12]

cols = ['shop_id', 'date_block_num']


for lag in lags:
    print(lag)
    lag_name = "shop_block_units_lag_" + str(lag)
    shop_block_units_lag = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['shop_id'],as_index=False)\
        ['shop_block_units'].shift(lag)\
        .rename(columns={'shop_block_units':lag_name}).reset_index()

    training = training.merge(shop_block_units_lag, on=cols, how='left')
    del shop_block_units_lag
    gc.collect()

1
2
3
6
12


In [ ]:
lags = [1,2,3,6,12]

cols = ['shop_id', 'item_id', 'date_block_num']


for lag in lags:
    print(lag)
    lag_name = "shop_item_block_units_lag_" + str(lag)
    shop_item_block_units_lag = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['shop_id', 'item_id'],as_index=False)\
        ['shop_item_block_units'].shift(lag)\
        .rename(columns={'shop_item_block_units':lag_name}).reset_index()

    training = training.merge(shop_item_block_units_lag, on=cols, how='left')
    del shop_item_block_units_lag
    gc.collect()

In [ ]:
lags = [1,2,3,6,12]

cols = ['shop_id', 'item_category_id', 'date_block_num']


for lag in lags:
    print(lag)
    lag_name = "shop_cat_block_units_lag_" + str(lag)
    shop_cat_block_units_lag = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['shop_id', 'item_category_id'],as_index=False)\
        ['shop_cat_block_units'].shift(lag)\
        .rename(columns={'shop_cat_block_units':lag_name}).reset_index()

    training = training.merge(shop_cat_block_units_lag, on=cols, how='left')
    del shop_cat_block_units_lag
    gc.collect()

In [ ]:
lags = [1,2,3,6,12]

cols = ['item_category_id', 'date_block_num']


for lag in lags:
    print(lag)
    lag_name = "cat_block_units_lag_" + str(lag)
    cat_block_units_lag = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_category_id'],as_index=False)\
        ['cat_block_units'].shift(lag)\
        .rename(columns={'cat_block_units':lag_name}).reset_index()

    training = training.merge(cat_block_units_lag, on=cols, how='left')
    del cat_block_units_lag
    gc.collect()

In [ ]:
str(int(0.25))

In [ ]:
training['rolling_composite'] =  training['shop_block_units_rolling_3'].clip(1, None) *\
            training['item_block_units_rolling_3'].clip(1, None) 
training['me_composite'] =  training['item_me'].clip(1, None)  * training['shop_me'].clip(1, None) 

In [70]:
training['comp_A'] = training['shop_block_units_lag_1'] * training['item_share_of_shop_units']

#training['shop_share_item_units_comp'] = training['item_units'] * training['shop_share_of_units']
training['comp_B'] = training['item_block_units_lag_1'] * training['item_share_of_shop_units']
training['shop_share_item_units_comp'] = training['item_units'] * training['shop_share_of_units']
training['shop_item_units_comp'] = training['item_units'] / training['shop_units']
training['comp_C'] = training['shop_block_units_lag_1'] * training['item_share_of_total_units']

#training['shop_share_item_units_comp'] = training['item_units'] * training['shop_share_of_units']
training['comp_D'] = training['item_block_units_lag_1'] * training['item_share_of_total_units']

training['comp_E'] = training['item_slope'] * training['item_block_units_lag_1']
#training['comp_F'] = training['shop_slope'] * training['shop_block_units_lag_1']
#training['comp_G'] = training['item_slope'] * training['shop_slope']
training['comp_H'] = training['item_slope'] * training['item_units']# * training['item_share_of_total_units']

training['comp_I'] = training['shop_item_block_units_mean_rolling_6'] * training['item_minmax_q0.25']




#'shop_share_item_units_comp', item shop
   # 'comp_A',  item shop
    #'comp_B',   item shop
   # 'comp_E',  item

In [260]:
def get_number_of_days_since_start(day,month, year):
    days = 0
    if year == 2015:
        days = 365
    def is_even(num):
        return num % 2 == 0
    half_of_month = int(month/2)
    even = (30*half_of_month) + (31*half_of_month)
    if is_even(month):
        days = days + even - 30 - day
    else:
        days = days + even + day
    return days

training['days_2_first_sale'] = training['item_id'].map(sales_train.set_index('item_id')\
    .apply(lambda row: get_number_of_days_since_start(row['day'],row['month'], row['year']),axis=1)\
    .groupby('item_id').min())
#transactions_items['item_days_since_start'] = transactions_items.apply(lambda row: get_number_of_days_since_start(row['day'],row['month'], row['year']),axis=1), downcast='unsigned') 





In [270]:
def get_number_of_days_since_start(day,month, year):
    days = 0
    if year == 2015:
        days = 365
    def is_even(num):
        return num % 2 == 0
    half_of_month = int(month/2)
    even = (30*half_of_month) + (31*half_of_month)
    if is_even(month):
        days = days + even - 30 - day
    else:
        days = days + even + day
    return days




def get_average_days_between_sales(days):
    days = sorted(np.unique(days))
    if len(days) == 0:
        return 9999
    if len(days) == 1:
        return 999
    return np.mean(np.ediff1d(days)) / len(days)



In [271]:
sales_train['item_days_since_start'] = sales_train\
    .apply(lambda row: get_number_of_days_since_start(row['day'],row['month'], row['year']),axis=1)
average_days_between_sales = sales_train.groupby(['item_id'])['item_days_since_start']\
    .apply(list).apply(lambda x: get_average_days_between_sales(x))

training['item_mean_day_between_activity'] = training['item_id'].map(average_days_between_sales)


In [277]:
sub_cats = {}
for i in range(1,8):
    sub_cats[i] = "Accessories"
sub_cats[8] = "Tickets"
sub_cats[9] = "Delivery of goods"
for i in range(10,18):
    sub_cats[i] = "Consoles"
for i in range(18,25):
    sub_cats[i] = "Game for Consoles"
sub_cats[25] = "Accessories for Games"
sub_cats[26] = "Android Games"
sub_cats[27] = "MAC Games"
for i in range(28,32):
    sub_cats[i] = "PC Games"
for i in range(32,37):
    sub_cats[i] = "Payment Cards"
for i in range(37,40):
    sub_cats[i] = "Cinema - Blu-ray"
sub_cats[40] = "Cinema - DVD"
sub_cats[41] = "Cinema - Collectible"
for i in range(42,46):
    sub_cats[i] = "Audiobooks"
for i in range(46,55):
    sub_cats[i] = "Books"
for i in range(55,57):
    sub_cats[i] = "Music - CD"
sub_cats[57] = "Music - MP3"
sub_cats[58] = "Music - Vinyl"
sub_cats[59] = "Music - Music Video"
sub_cats[60] = "Music - Gift Edition"
for i in range(61,74):
    sub_cats[i] = "Gifts"
for i in range(73,79):
    sub_cats[i] = "Software"
sub_cats[79] = "Utility"
for i in range(80,84):
    sub_cats[i] = "Misc"
    
    
training['subcategory'] = training['item_category_id'].apply(lambda x: sub_cats[x]).astype('category')
training['subcategory_me'] = get_mean_encoding(training, ['subcategory'], 'y')


In [281]:
shop_areas = {}
for i in range(0,2):
    shop_areas[i] = "Yakutsk"
shop_areas[2] = "Adygea"
shop_areas[3] = "Balashikha"
shop_areas[4] = "Volga"
shop_areas[5] = "Vologda"
for i in range(6,9):
    shop_areas[i] = "Voronezh"
shop_areas[9] = "Outbound Trading"
for i in range(10,12):
    shop_areas[i] = "Zhukovsky"
shop_areas[12] = "Online store emergency"
for i in range(13,15):
    shop_areas[i] = "Kazan"
shop_areas[15] = "Kaluga"
shop_areas[16] = "Kolomna"
for i in range(17,19):
    shop_areas[i] = "Krasnoyarsk"
shop_areas[19] = "Kursk"
for i in range(20,33):
    shop_areas[i] = "Moscow"
shop_areas[33] = "Mytishchi"
for i in range(34,36):
    shop_areas[i] = "N.Novgorod"
for i in range(36,38):
    shop_areas[i] = "Novosibirsk"
shop_areas[38] = "Omsk"
for i in range(39,42):
    shop_areas[i] = "RostovNaDonu"
for i in range(42,44):
    shop_areas[i] = "St. Petersburg"
for i in range(44,46):
    shop_areas[i] = "Samara"
shop_areas[46] = "Sergiev Posad"
shop_areas[47] = "Surgut"
shop_areas[48] = "Tomsk"
for i in range(49,52):
    shop_areas[i] = "Tyumen TC"
for i in range(52,54):
    shop_areas[i] = "Ufa"
shop_areas[54] = "Khimki"
shop_areas[55] = "Digital warehouse"
shop_areas[56] = "Chekhov"
for i in range(57,59):
    shop_areas[i] = "Yakutsk"
shop_areas[59] = "Yaroslavl"

training['area'] = training['shop_id'].apply(lambda x: shop_areas[x]).astype('category')
training['area_me'] = get_mean_encoding(training, ['area'], 'y')


In [61]:
training.fillna(0, inplace=True)
for column in training.columns.values:
    if "units" in column and "share" not in column:
        training[column] = pd.to_numeric(training[column].astype(int), downcast='unsigned')
    else:
        training[column] = pd.to_numeric(training[column], downcast='float')


In [71]:
gc.collect()

1226

In [30]:
training.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6425094 entries, 0 to 6425093
Data columns (total 71 columns):
item_id                           float32
shop_id                           float32
date_block_num                    float32
y                                 int8
item_category_id                  float32
item_me                           float32
shop_me                           float32
category_me                       float32
shop_category_me                  float32
shop_item_me                      float32
item_block_units                  int64
shop_block_units                  int64
cat_block_units                   int64
shop_cat_block_units              int64
shop_item_block_units             int64
item_units                        uint32
item_max_units_block              int64
item_min_units_block              int64
item_minmax_mean                  float32
item_minmax_q0.25                 float32
item_minmax_q0.5                  float32
item_minmax_q0.75      

In [ ]:
training.dtypes

In [ ]:
training.sample(10)

In [ ]:
training[(training['item_id'].isin([30,31])) & (training['shop_id'] == 30)]\
        .sort_values(['item_id','date_block_num'])[['item_id','shop_id',\
                                                    'date_block_num','item_block_units', 'item_block_units_lag_1',\
                                                    'item_block_units_lag_2','item_block_units_lag_3',\
                                                    'item_block_units_lag_6','item_block_units_lag_12'
                                                   ]]
                                                    #'item_block_units_rolling_3', 'item_block_units_rolling_6']]
                                                    #'item_block_units_rolling_6']]

In [ ]:
len(training[training['item_block_units'] > 0])

In [ ]:
len(transactions_items_blocks)

In [ ]:
gc.collect()
val = training[training['date_block_num'] == 33]
print("val length", len(val))

unique_pairs_val = list(set(list(zip(val.shop_id, val.item_id))))
print("number of unique shop/item pairs in val", len(unique_pairs_val))
unique_pairs_val_ignore = unique_pairs_val[0:int(len(unique_pairs_val)/2)]


def tuple2key(t):
    return "%d_%d" % (t[0], t[1])

val_pairs_ignore_dict = {}
for t in unique_pairs_val_ignore:
    val_pairs_ignore_dict[tuple2key(t)] = 1
 
    
training['val_ignore'] = (training['shop_id'].astype(str) + '_' +  training['item_id'].astype(str))\
                                    .apply(lambda x: x in val_pairs_ignore_dict)

In [25]:
def get_coeffs_for_item(df, item_id, poly_degree=1, print_chart=False):

    cat = df[df.item_id == item_id].groupby('date_block_num')['item_cnt_day'].sum().reset_index(name ='item_cnt')

    datax = cat.date_block_num.values
    datay = cat.item_cnt.values
    if len(datay) == 0:
        return [0]
    
    z = np.polyfit(datax,datay, poly_degree) 
    p = np.poly1d(z)

    cat['poly'] = cat['date_block_num'].apply(lambda x: p(x))

    if print_chart:
        sns.set()
        fig, ax = plt.subplots()
        cat.plot(x='date_block_num', y='item_cnt', ax=ax, legend=False, figsize=(15,7))
        cat.plot(x='date_block_num', y='poly', ax=ax, legend=False, figsize=(15,7))

    return p

slopes = {}

for item_id in all_item_ids:
    slope = get_coeffs_for_item(sales_train,item_id)[0]
    slopes[item_id] = slope

training['item_slope'] = training.apply(lambda row: slopes[row['item_id']], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr

...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.


In [69]:
def get_coeffs_for_shop(df, item_id, poly_degree=1, print_chart=False):

    cat = df[df.shop_id == shop_id].groupby('date_block_num')['item_cnt_day'].sum().reset_index(name ='item_cnt')

    datax = cat.date_block_num.values
    datay = cat.item_cnt.values
    if len(datay) == 0:
        return [0]
    
    z = np.polyfit(datax,datay, poly_degree) 
    p = np.poly1d(z)

    cat['poly'] = cat['date_block_num'].apply(lambda x: p(x))

    if print_chart:
        sns.set()
        fig, ax = plt.subplots()
        cat.plot(x='date_block_num', y='item_cnt', ax=ax, legend=False, figsize=(15,7))
        cat.plot(x='date_block_num', y='poly', ax=ax, legend=False, figsize=(15,7))

    return p

slopes = {}

for shop_id in all_shop_ids:
    slope = get_coeffs_for_shop(sales_train,shop_id)[0]
    slopes[shop_id] = slope

training['shop_slope'] = training.apply(lambda row: slopes[row['shop_id']], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
len(training[training['val_ignore'] == True])

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5).fit(training[cb_features].fillna(0))

In [ ]:
training = pca.transform(training[cb_features].fillna(0))

In [ ]:
training

In [71]:
gc.collect()

ZEROS_KEEP=1.75


#x_train = training[(training['date_block_num'] < 33) & (training['val_ignore'] == False)]
x_train = training[(training['date_block_num'] < 33)]
y_train = x_train['y']


pos_train_len = len(y_train[y_train != 0])
print("pos_train_len", pos_train_len)

zeros_keep_indices_train = y_train[y_train == 0].sample(int(pos_train_len/ZEROS_KEEP)).index
print("zeros_keep_indices_train", len(zeros_keep_indices_train))
non_zeros_train_indices = y_train[y_train != 0].index
print("non_zeros_train_indices", len(non_zeros_train_indices))

train_indices = np.append(np.array(zeros_keep_indices_train), np.array(non_zeros_train_indices))

#y_train = y_train.loc[train_indices]
#x_train = x_train.loc[train_indices]




x_val = training[training['date_block_num'] == 33]
y_val = x_val['y']

pos_val_len = len(y_val[y_val != 0])
print("pos_val_len", pos_val_len)

zeros_keep_indices_val = y_val[y_val == 0].sample(int(pos_val_len/ZEROS_KEEP)).index
print("zeros_keep_indices_val", len(zeros_keep_indices_val))
non_zeros_val_indices = y_val[y_val != 0].index
print("non_zeros_val_indices", len(non_zeros_val_indices))

val_indices = np.append(np.array(zeros_keep_indices_val), np.array(non_zeros_val_indices))

y_val = y_val.loc[val_indices]
x_val = x_val.loc[val_indices]


pos_train_len 887869
zeros_keep_indices_train 507353
non_zeros_train_indices 887869
pos_val_len 31471
zeros_keep_indices_val 17983
non_zeros_val_indices 31471


In [69]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'y', 'item_category_id',
       'item_me', 'shop_me', 'category_me', 'shop_category_me',
       'shop_item_me', 'item_block_units', 'shop_block_units',
       'cat_block_units', 'shop_cat_block_units', 'shop_item_block_units',
       'item_units', 'item_max_units_block', 'item_min_units_block',
       'item_minmax_mean', 'item_minmax_q0.25', 'item_minmax_q0.5',
       'item_minmax_q0.75', 'shop_units', 'shop_max_units_block',
       'shop_min_units_block', 'shop_minmax_mean', 'shop_minmax_q0.25',
       'shop_minmax_q0.5', 'shop_minmax_q0.75', 'cat_units',
       'cat_max_units_block', 'cat_min_units_block', 'cat_minmax_mean',
       'shop_cat_units', 'shop_cat_max_units_block',
       'shop_cat_min_units_block', 'shop_cat_minmax_mean',
       'shop_item_units_x', 'shop_item_max_units_block',
       'shop_item_min_units_block', 'shop_item_minmax_mean',
       'item_share_of_total_units', 'category_share_of_total_units',
       'shop_share_

In [81]:



features = [
       'item_me', 'shop_me', 'category_me', 'shop_category_me',
    'item_units',
'item_minmax_q0.25', 'item_minmax_q0.5',
       'item_minmax_q0.75',
    'item_block_units_rolling_2', 'item_block_units_rolling_12',
     'item_block_units_lag_1', 'shop_share_item_units_comp',
    'comp_A', 'comp_B',
    'comp_E',
    'shop_item_block_units_mean_rolling_6',
]

In [82]:

gc.collect()
params =   {
    'objective' : 'reg:linear',
    'tree_method':'gpu_hist',
    #'gpu_id': 0,
    'learning_rate': 0.05, 
    #'gamma' : 0.3, 
    #'min_child_weight' : 3,
    #'nthread' : 16,
    #'max_depth' : 7,
    #'subsample' : 0.7, 
    #'colsample_bytree' : 0.7, 
    'seed':42, 
    'eval_metric' : "rmse",
    'num_boost_round' : 70000,
    #'n_estimators':999,
    #'max_leaves': 300
}


tr_data = xgb.DMatrix(x_train[features], y_train)
va_data = xgb.DMatrix(x_val[features], y_val)


watchlist = [(tr_data, 'train'), (va_data, 'valid')]

xg_model = xgb.train(params, tr_data, 700, watchlist, maximize=False, early_stopping_rounds = 10, verbose_eval=10)

[0]	train-rmse:1.17325	valid-rmse:2.30053
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[10]	train-rmse:0.941041	valid-rmse:1.93287
[20]	train-rmse:0.828318	valid-rmse:1.75312
[30]	train-rmse:0.773681	valid-rmse:1.65579
[40]	train-rmse:0.744634	valid-rmse:1.59322
[50]	train-rmse:0.728058	valid-rmse:1.55275
[60]	train-rmse:0.718572	valid-rmse:1.52645
[70]	train-rmse:0.711609	valid-rmse:1.50551
[80]	train-rmse:0.706484	valid-rmse:1.49423
[90]	train-rmse:0.702392	valid-rmse:1.47898
[100]	train-rmse:0.699167	valid-rmse:1.47061
[110]	train-rmse:0.696127	valid-rmse:1.46235
[120]	train-rmse:0.692916	valid-rmse:1.45662
[130]	train-rmse:0.689424	valid-rmse:1.45137
[140]	train-rmse:0.686645	valid-rmse:1.44513
[150]	train-rmse:0.684088	valid-rmse:1.44017
[160]	train-rmse:0.681261	valid-rmse:1.43881
[170]	train-rmse:0.678574	valid-rmse:1.43548
[180]	train-rmse:0.676533	valid-rmse:1.43476
[190]	train-

In [142]:

features = [
       'item_me', 'shop_me', 'category_me', 'shop_category_me',
    'item_units',#'item_share_of_total_units',
'item_minmax_q0.25', 'item_minmax_q0.5',
       'item_minmax_q0.75',
    'item_block_units_rolling_2', 'item_block_units_rolling_12',
     'item_block_units_lag_1', 'shop_share_item_units_comp',
    'comp_A', 'comp_B',
    'comp_E',
]

In [113]:

item_features+merge_col

['item_me',
 'item_units',
 'item_minmax_q0.25',
 'item_minmax_q0.5',
 'item_minmax_q0.75',
 'item_id']

In [169]:
item_features = [ 'item_me', 
    'item_units',
'item_minmax_q0.25', 'item_minmax_q0.5',
       'item_minmax_q0.75', 'comp_E'
    ]

merge_col = ['item_id']
cols=item_features+merge_col

test = test.merge(training.drop_duplicates('item_id')[cols], on=merge_col, how='left')

In [170]:
shop_features = [
        'shop_me'
]

merge_col = ['shop_id']
cols=shop_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [171]:
test = test.set_index('item_id').join(items.set_index('item_id'))
test.reset_index(inplace=True)

In [172]:
cat_features = [
              'category_me',
]

merge_col = ['item_category_id']
cols=cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [173]:
shop_item_features = [
'shop_share_item_units_comp', 
   'comp_A',  
    'comp_B'
]

merge_col = ['shop_id', 'item_id']
cols=shop_item_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [174]:
shop_cat_features = [
'shop_category_me'
]

merge_col = ['shop_id', 'item_category_id']
cols=shop_cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [175]:
lags = [1]

cols = ['item_id', 'date_block_num']


for lag in lags:
    print(lag)
    lag_name = "item_block_units_lag_" + str(lag)
    item_block_units_lag = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_id'],as_index=False)\
        ['item_block_units'].shift(lag)\
        .rename(columns={'item_block_units':lag_name}).reset_index()

     
    thirty_three = item_block_units_lag[item_block_units_lag['date_block_num'] == 33].drop_duplicates(cols)\
                [['item_id', lag_name]]
    test = test.merge(thirty_three, on=['item_id'], how='left')

1


In [103]:
test['date_block_num'] = 34

In [176]:
rolls = [2,12]
cols = ['item_id','date_block_num']

for roll in rolls:
    print(roll)
    roll_name = "item_block_units_rolling_" + str(roll)
    roll_name_tmp = roll_name + "_tmp"
    
    item_block_units_rolling_temp = training\
        .drop_duplicates(cols)\
        .sort_values(cols)\
        .set_index(cols)\
        .groupby(['item_id'],as_index=False)\
        ['item_block_units'].rolling(roll,min_periods=2).mean().reset_index()\
        .rename(columns={'item_block_units':roll_name})\
        [['item_id','date_block_num',roll_name]]
    
    thirty_three = item_block_units_rolling_temp[item_block_units_rolling_temp['date_block_num'] == 33].drop_duplicates(cols)\
                [['item_id', roll_name]]
    test = test.merge(thirty_three, on=['item_id'], how='left')

2
12


In [165]:
test.fillna(0, inplace=True)

In [167]:
test.sample(10)

,item_id,ID,shop_id,item_me,item_units,item_minmax_q0.25,item_minmax_q0.5,item_minmax_q0.75,comp_E,shop_me,item_category_id,category_me,shop_share_item_units_comp,comp_A,comp_B,item_block_units_lag_1_x,shop_category_me,item_block_units_lag_1_y,item_block_units_rolling_2,item_block_units_rolling_12
152871,15444,171758,37,0.0,5377354.0,6.50,11.0,15.50,0.0,0.0,63,0.0,2.276037e+11,0.000000,0.0,4.0,0.0,4.0,5.0,9.416667
13239,1573,50598,31,0.0,2756902.0,1.75,2.5,3.25,-0.0,0.0,31,0.0,4.420865e+10,0.000000,0.0,4.0,0.0,4.0,4.0,3.250000
9591,1290,79575,15,0.0,3657869.0,8.25,11.5,14.75,0.0,0.0,55,0.0,7.475105e+10,0.000325,0.0,6.0,0.0,6.0,6.0,8.714286
142440,14532,92189,14,0.0,5377354.0,3.75,6.5,9.25,0.0,0.0,55,0.0,2.276037e+11,0.000000,0.0,2.0,0.0,2.0,2.5,4.333333
40517,4141,149480,59,0.0,3132249.0,1.50,2.0,2.50,-0.0,0.0,75,0.0,5.563274e+10,0.000000,0.0,1.0,0.0,1.0,2.0,2.000000
109938,11523,124603,52,0.0,5377354.0,8.50,16.0,23.50,-0.0,0.0,70,0.0,2.276037e+11,0.000000,0.0,1.0,0.0,1.0,16.0,9.444444
207714,21388,126373,52,0.0,5892250.0,20.25,39.5,58.75,0.0,0.0,37,0.0,2.104952e+11,0.000164,0.0,2.0,0.0,2.0,1.5,20.777778
107638,11267,176729,35,0.0,0.0,0.00,0.0,0.00,0.0,0.0,40,0.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
191843,19656,148210,59,0.0,3126464.0,57.00,57.0,57.00,0.0,0.0,38,0.0,5.766173e+10,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
191437,19621,7360,4,0.0,4564000.0,5.50,10.0,14.50,0.0,0.0,40,0.0,1.320638e+11,0.000402,0.0,2.0,0.0,2.0,1.5,6.750000


In [190]:
cb_model = CatBoostRegressor(iterations=250,
                             learning_rate=0.001,
                             eval_metric='RMSE',
                             #thread_count=16,
                             task_type = "GPU",
                             use_best_model=True,
                             #l2_leaf_reg = 1000,
                             od_type = "Iter",
                             od_wait = 30,
                             #random_strength = 10,
                             #bagging_temperature = 1,
                             #one_hot_max_size = 2,
                             random_seed = 42)

#drops = ['subcategory','area']
#x_train = x_train.drop(columns=drops)
#x_val = x_val.drop(columns=drops)


cb_model.fit(x_train[features], y_train, #cat_features=categorical_features_indices,
             eval_set=(x_val[features],y_val),
             #cat_features=categorical_features_pos,         
             verbose=True)

0:	learn: 1.2222963	test: 1.1647238	best: 1.1647238 (0)	total: 41.5ms	remaining: 10.3s
1:	learn: 1.2217218	test: 1.1642269	best: 1.1642269 (1)	total: 84.1ms	remaining: 10.4s
2:	learn: 1.2211485	test: 1.1637477	best: 1.1637477 (2)	total: 130ms	remaining: 10.7s
3:	learn: 1.2205759	test: 1.1632728	best: 1.1632728 (3)	total: 175ms	remaining: 10.8s
4:	learn: 1.2199868	test: 1.1627954	best: 1.1627954 (4)	total: 217ms	remaining: 10.6s
5:	learn: 1.2194189	test: 1.1623042	best: 1.1623042 (5)	total: 258ms	remaining: 10.5s
6:	learn: 1.2188305	test: 1.1618461	best: 1.1618461 (6)	total: 302ms	remaining: 10.5s
7:	learn: 1.2182444	test: 1.1613891	best: 1.1613891 (7)	total: 343ms	remaining: 10.4s
8:	learn: 1.2176773	test: 1.1609134	best: 1.1609134 (8)	total: 387ms	remaining: 10.4s
9:	learn: 1.2170862	test: 1.1604229	best: 1.1604229 (9)	total: 428ms	remaining: 10.3s
10:	learn: 1.2165202	test: 1.1599490	best: 1.1599490 (10)	total: 471ms	remaining: 10.2s
11:	learn: 1.2159785	test: 1.1594885	best: 1.15948

95:	learn: 1.1704036	test: 1.1221654	best: 1.1221654 (95)	total: 3.94s	remaining: 6.33s
96:	learn: 1.1699019	test: 1.1217334	best: 1.1217334 (96)	total: 3.99s	remaining: 6.29s
97:	learn: 1.1693819	test: 1.1212940	best: 1.1212940 (97)	total: 4.03s	remaining: 6.25s
98:	learn: 1.1688603	test: 1.1208596	best: 1.1208596 (98)	total: 4.07s	remaining: 6.2s
99:	learn: 1.1683390	test: 1.1204292	best: 1.1204292 (99)	total: 4.11s	remaining: 6.16s
100:	learn: 1.1678367	test: 1.1200151	best: 1.1200151 (100)	total: 4.15s	remaining: 6.12s
101:	learn: 1.1673505	test: 1.1195902	best: 1.1195902 (101)	total: 4.19s	remaining: 6.08s
102:	learn: 1.1668341	test: 1.1191765	best: 1.1191765 (102)	total: 4.23s	remaining: 6.04s
103:	learn: 1.1663208	test: 1.1187256	best: 1.1187256 (103)	total: 4.27s	remaining: 6s
104:	learn: 1.1658211	test: 1.1183153	best: 1.1183153 (104)	total: 4.31s	remaining: 5.95s
105:	learn: 1.1653074	test: 1.1178954	best: 1.1178954 (105)	total: 4.35s	remaining: 5.91s
106:	learn: 1.1648123	te

188:	learn: 1.1255089	test: 1.0853111	best: 1.0853111 (188)	total: 7.7s	remaining: 2.48s
189:	learn: 1.1250712	test: 1.0849692	best: 1.0849692 (189)	total: 7.74s	remaining: 2.44s
190:	learn: 1.1246110	test: 1.0845915	best: 1.0845915 (190)	total: 7.78s	remaining: 2.4s
191:	learn: 1.1241691	test: 1.0842104	best: 1.0842104 (191)	total: 7.82s	remaining: 2.36s
192:	learn: 1.1237289	test: 1.0838409	best: 1.0838409 (192)	total: 7.86s	remaining: 2.32s
193:	learn: 1.1232923	test: 1.0834997	best: 1.0834997 (193)	total: 7.9s	remaining: 2.28s
194:	learn: 1.1228513	test: 1.0831143	best: 1.0831143 (194)	total: 7.94s	remaining: 2.24s
195:	learn: 1.1224125	test: 1.0827240	best: 1.0827240 (195)	total: 7.98s	remaining: 2.2s
196:	learn: 1.1219570	test: 1.0823579	best: 1.0823579 (196)	total: 8.02s	remaining: 2.16s
197:	learn: 1.1215133	test: 1.0819837	best: 1.0819837 (197)	total: 8.06s	remaining: 2.12s
198:	learn: 1.1210757	test: 1.0815911	best: 1.0815911 (198)	total: 8.1s	remaining: 2.08s
199:	learn: 1.1

In [ ]:
#training.to_pickle("pickled/training")
#training = pd.read_pickle("pickled/training")

#pickle.dump(cb_model, open( "pickled/cb_model", "wb"), protocol=4)

#cb_model = pickle.load( open( "pickled/cb_model", "rb" ) )

In [ ]:
scores = {}
for i,score in enumerate(cb_model.get_feature_importance()):
    scores[cb_features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

In [ ]:
cb_features = [item[0] for item in scores.items() if item[1] > 4]

In [78]:
scores = xg_model.get_score(importance_type='gain')


sorted(scores.items(), key=lambda x: x[1])[::-1]

[('comp_B', 9975.585759605745),
 ('comp_A', 3500.096114185682),
 ('item_minmax_q0.75', 2539.406513239468),
 ('item_minmax_q0.5', 2475.0095673973683),
 ('item_block_units_lag_1', 2215.4428950527436),
 ('item_block_units_rolling_12', 1860.2978936653512),
 ('shop_share_item_units_comp', 1737.2460632432787),
 ('item_minmax_q0.25', 711.5185389825049),
 ('shop_category_me', 647.1026757273506),
 ('item_me', 564.3782301505202),
 ('category_me', 559.1599217124464),
 ('item_units', 553.3614927434423),
 ('shop_item_block_units_mean_rolling_6', 447.70870001619437),
 ('shop_me', 385.7073280005373),
 ('comp_E', 349.74763761839193),
 ('item_block_units_rolling_2', 295.62211615872036)]

In [181]:
xg_preds = xg_model.predict(xgb.DMatrix(test[features]))
xg_preds.clip(0,20,out=xg_preds)

array([0.30507183, 0.30507183, 0.30507183, ..., 0.3088768 , 0.4178506 ,
       0.3088768 ], dtype=float32)

In [186]:
cb_preds = cb_model.predict(test[features])
cb_preds.clip(0,20,out=cb_preds)

array([0.02456795, 0.02456795, 0.02456795, ..., 0.02658548, 0.0890402 ,
       0.02658548])

In [188]:
np.max(cb_preds)

5.507234866730869

In [189]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = cb_preds.astype(int)
#submission['item_cnt_month'] = ensemble_preds.astype(int)


submission.to_csv('submission.csv', index=False)